In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, dayofweek, month, radians, asin, sqrt, pow, dayofyear, log, exp, sin, cos, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
spark = SparkSession.builder \
    .appName("Problem2 Structured API") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()


25/04/11 18:50:40 WARN Utils: Your hostname, nam-Nitro-AN515-45 resolves to a loopback address: 127.0.1.1; using 192.168.1.18 instead (on interface wlp5s0)
25/04/11 18:50:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 18:50:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load data
try:
    train_data = spark.read.csv("train.csv", header=True, inferSchema=True)
except Exception as e:
    print(f"Error loading train data: {e}")
    spark.stop()
    exit(1)

train_data.show()

+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1| -73.9821548461914| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
|id2377394|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.98041534423828|40.738563537597656|-73.99948120117188| 40.73115158081055|                 N|          663|
|id3858529|        2|2016-01-19 11:35:24|2016-01-19 12:10:48|    

In [4]:
# Feature engineering function
def prepare_features(data):
    return data.withColumn("pickup_day_of_year", dayofyear("pickup_datetime")) \
        .withColumn("pickup_day_of_week", dayofweek("pickup_datetime")) \
        .withColumn("pickup_hour_of_day", hour("pickup_datetime")) \
        .withColumn("pickup_month", month("pickup_datetime")) \
        .withColumn("haversine",
                    6371 * 2 * asin(sqrt(pow(sin((radians("pickup_latitude") - radians("dropoff_latitude")) / 2), 2) +
                                         cos(radians("pickup_latitude")) * cos(radians("dropoff_latitude")) *
                                         pow(sin((radians("pickup_longitude") - radians("dropoff_longitude")) / 2),
                                             2))))


# Data cleaning and outlier removal using IQR
def remove_outliers(data, column, lower_quantile=0.25, upper_quantile=0.75, k=1.5):
    quantiles = data.approxQuantile(column, [lower_quantile, upper_quantile], 0.05)
    iqr = quantiles[1] - quantiles[0]
    lower_bound = quantiles[0] - k * iqr
    upper_bound = quantiles[1] + k * iqr
    return data.filter((col(column) >= lower_bound) & (col(column) <= upper_bound))


# Evaluate model
def evaluate_model(predictions, dataset_name):
    predictions = predictions.withColumn("predicted_duration", col("prediction"))
    metrics = {}
    for metric in ["rmse", "mse", "mae", "r2"]:
        evaluator = RegressionEvaluator(labelCol="trip_duration", predictionCol="predicted_duration", metricName=metric)
        value = evaluator.evaluate(predictions)
        metrics[metric] = value
        print(f"\n{metric.upper()} on {dataset_name} = {value}")
    return metrics

In [5]:
# Features
train_data = prepare_features(train_data)
train_data.cache()

# Remove outliers
train_data = train_data.filter("haversine > 0").filter("passenger_count > 0").filter("trip_duration > 0")
train_data = remove_outliers(train_data, "trip_duration")
train_data = remove_outliers(train_data, "haversine")

25/04/11 18:50:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
features = ["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude",
            "pickup_day_of_year", "pickup_day_of_week", "pickup_hour_of_day", "pickup_month",
            "haversine", "passenger_count"]

# VectorAssembler
assembler = VectorAssembler(inputCols=features, outputCol="features")
assembled_data = assembler.transform(train_data).select("id", "features", "trip_duration")
assembled_data.cache()

DataFrame[id: string, features: vector, trip_duration: int]

In [7]:
# Define Decision Tree Regressor
decision_tree = DecisionTreeRegressor(featuresCol="features", labelCol="trip_duration", seed=42)

# Define parameter grid for hyperparameter tuning
param_grid = ParamGridBuilder() \
    .addGrid(decision_tree.maxDepth, [5, 7, 10]) \
    .addGrid(decision_tree.minInstancesPerNode, [3, 5, 7, 10]) \
    .build()

# Define evaluator
evaluator = RegressionEvaluator(labelCol="trip_duration", predictionCol="prediction", metricName="rmse")

# Set up CrossValidator
crossval = CrossValidator(estimator=decision_tree,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5,
                          seed=42)

In [8]:
# Train model
try:
    cv_model = crossval.fit(assembled_data)
    best_model = cv_model.bestModel
except Exception as e:
    print(f"Error during model training: {e}")
    spark.stop()
    exit(1)

# Split into training and validation sets for final evaluation
train, validation = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [9]:
# Predictions and evaluation
train_predictions = best_model.transform(train)
print("\n=== Training Data Metrics ===")
train_metrics = evaluate_model(train_predictions, "training data")

validation_predictions = best_model.transform(validation)
print("\n=== Validation Data Metrics ===")
validation_metrics = evaluate_model(validation_predictions, "validation data")


=== Training Data Metrics ===



RMSE on training data = 234.49713476712958



MSE on training data = 54988.90621399333

MAE on training data = 172.71133268041575

R2 on training data = 0.598813666260533

=== Validation Data Metrics ===

RMSE on validation data = 235.1705973245371

MSE on validation data = 55305.20984597958

MAE on validation data = 173.23089203901273

R2 on validation data = 0.5992673263373948


In [10]:
# Model diagnostics
print("\n=== Model Diagnostics ===")
print("Feature Importances:")
for i, importance in enumerate(best_model.featureImportances):
    print(f"{features[i]}: {importance}")

print("Decision Tree Depth:", best_model.depth)
print("Number of Nodes:", best_model.numNodes)
print("Number of Features:", best_model.numFeatures)
print("Best maxDepth:", best_model.getMaxDepth())
print("Best minInstancesPerNode:", best_model.getMinInstancesPerNode())

train_data.unpersist()
assembled_data.unpersist()
spark.stop()


=== Model Diagnostics ===
Feature Importances:
pickup_longitude: 0.00385487018403132
pickup_latitude: 0.0033164470727624074
dropoff_longitude: 0.006813564675981587
dropoff_latitude: 0.05450252495062818
pickup_day_of_year: 0.0010952752520476266
pickup_day_of_week: 0.0472797813520864
pickup_hour_of_day: 0.11019161468908256
pickup_month: 8.211604121482907e-06
haversine: 0.7728617931490022
passenger_count: 7.59170702563842e-05
Decision Tree Depth: 10
Number of Nodes: 1971
Number of Features: 10
Best maxDepth: 10
Best minInstancesPerNode: 10
